In [1]:
#Load packages
import pandas as pd
import numpy as np
from mplsoccer import Pitch, Sbopen
from statsbombpy import sb

In [2]:
# Find IDs for La Liga 2015/16 
competitions = sb.competitions()
competitions = competitions[(competitions['country_name'] == 'Spain') & (competitions['season_name'] == '2015/2016')]
competitions.sort_values(by='season_name', ascending=False).head()

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
43,11,27,Spain,La Liga,male,False,False,2015/2016,2024-05-16T14:06:52.149840,2021-06-13T16:17:31.694,None,2024-05-16T14:06:52.149840


In [3]:
# Create list of Match IDs
LaLiga201516 = sb.matches(competition_id=11, season_id=27)
match_id_list = LaLiga201516['match_id'].tolist()

In [6]:
# Load the xT grid data
xT = pd.read_csv("/Users/julianb/Downloads/xT_grid.csv")
xT = np.array(xT)
xT_rows, xT_cols = xT.shape

In [9]:
# Initialize the parser
parser = Sbopen()

# Initialize an empty list to store the results
all_match_xT = []

# Loop over each match in La Liga 2015/16
for match_id in match_id_list:
    # Fetch the events data for each match (df, related, freeze, tactics)
    df, related, freeze, tactics = parser.event(match_id)
    
    # Filter the DataFrame by selecting only the specified columns
    df = df[['period', 'minute', 'second', 'team_name', 'x', 'y', 'player_name', 'end_x', 'end_y', 'type_name', 'outcome_name']]
    
    # Filter for passes (where type_name is 'Pass')
    df_pass = df[(df['type_name'] == 'Pass') & (df['outcome_name'].isna())]

    # Bin Data for xT: Create integer locators for each bin in the grid
    df_pass['x1_bin'] = pd.cut(df_pass['x'], bins=xT_cols, labels=False)
    df_pass['y1_bin'] = pd.cut(df_pass['y'], bins=xT_rows, labels=False)
    df_pass['x2_bin'] = pd.cut(df_pass['end_x'], bins=xT_cols, labels=False)
    df_pass['y2_bin'] = pd.cut(df_pass['end_y'], bins=xT_rows, labels=False)
    
    # Calculate the xT value for each pass
    df_pass['start_zone_value'] = df_pass[['x1_bin', 'y1_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
    df_pass['end_zone_value'] = df_pass[['x2_bin', 'y2_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
    df_pass['xT'] = df_pass['end_zone_value'] - df_pass['start_zone_value']

    # Get the home and away teams, score and gaol difference for this match
    home_team = LaLiga201516.loc[LaLiga201516['match_id'] == match_id, 'home_team'].values[0]
    away_team = LaLiga201516.loc[LaLiga201516['match_id'] == match_id, 'away_team'].values[0]
    home_score = LaLiga201516.loc[LaLiga201516['match_id'] == match_id, 'home_score'].values[0]
    away_score = LaLiga201516.loc[LaLiga201516['match_id'] == match_id, 'away_score'].values[0]
    goal_difference = home_score - away_score

    # Create home_xT, away_xT and difference by summing the xT for each team
    home_xT_sum = df_pass[df_pass['team_name'] == home_team]['xT'].sum()
    away_xT_sum = df_pass[df_pass['team_name'] == away_team]['xT'].sum()
    xT_difference = home_xT_sum - away_xT_sum

    # Store the results in a dictionary
    match_xT_data = {
        'match_id': match_id,
        'home_team': home_team,
        'away_team': away_team,
        'home_xT': home_xT_sum,
        'away_xT': away_xT_sum,
        'home_score': home_score, 
        'away_score': away_score,
        'xT_difference': xT_difference,
        'goal_difference': goal_difference
    }

    # Append the match data to the list
    all_match_xT.append(match_xT_data)

# Convert the list of match xT data to a DataFrame
final_xT_results = pd.DataFrame(all_match_xT)

/var/folders/q_/3dw4jg3d1zg1vtq9c1d082h00000gn/T/ipykernel_54670/643208470.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pass['x1_bin'] = pd.cut(df_pass['x'], bins=xT_cols, labels=False)
/var/folders/q_/3dw4jg3d1zg1vtq9c1d082h00000gn/T/ipykernel_54670/643208470.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pass['y1_bin'] = pd.cut(df_pass['y'], bins=xT_rows, labels=False)
/var/folders/q_/3dw4jg3d1zg1vtq9c1d082h00000gn/T/ipykernel_54670/643208470.py:21: SettingWithCopyWarning: 
A value is 

In [11]:
# Display the final result with only the columns neccessary for ML modeling 
final_xT_results

,match_id,home_team,away_team,home_xT,away_xT,home_score,away_score,xT_difference,goal_difference
0,3825848,Levante UD,Eibar,2.445716,1.566409,2,2,0.879307,0
1,3825895,Las Palmas,Sevilla,0.953666,2.813332,2,0,-1.859666,2
2,3825894,RC Deportivo La Coruña,Getafe,1.560558,1.185853,0,2,0.374705,-2
3,3825855,Málaga,Levante UD,3.367956,2.352049,3,1,1.015907,2
4,3825908,Espanyol,Eibar,1.334277,1.317793,4,2,0.016485,2
...,...,...,...,...,...,...,...,...,...
375,266467,Celta Vigo,Barcelona,0.893834,1.755540,4,1,-0.861707,3
376,267273,Las Palmas,Barcelona,1.206697,1.950168,1,2,-0.743471,-1
377,266490,Barcelona,Levante UD,1.957281,0.836360,4,1,1.120920,3
378,266254,Barcelona,Celta Vigo,2.465494,1.245869,6,1,1.219625,5
